In [ ]:
!pip install ollama

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
import scipy
import sympy
from sympy import *
from sympy.solvers.ode.systems import dsolve_system
from sympy import init_printing
init_printing()
from sympy import symbols, Eq, Function, dsolve
from sympy import init_printing
init_printing()
import math
from scipy.integrate import odeint
from scipy.integrate import odeint
from scipy.integrate import solve_ivp  # or nothing
import os
import random
import torch
import json
import requests
import time
import hashlib
from typing import List, Dict, Any, Tuple, Literal, Union
from tqdm import tqdm
import ast
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
#from x_llm import llm_query
# import ollama
import weakref
from google.colab import drive
import sys
import io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
DEFAULT_MODEL_PARAMS = {'top_k':1,'temperature':0.1, 'top_p':0.0001}

# CACHE_PATH = os.path.join( 'tmp', "cache_llm_respostas_ANALISES.json" )
#CACHE_PATH = "codigo_python/cache_llm_respostas_ANALISES.json"
CACHE_PATH = "cache_llm_respostas_ANALISES.json"
HOST_OLLAMA = "10.10.10.106:11434"
HOST_OLLAMA = 'local'
SAVE_CACHE_INTERVAL=1
SEED = 42
DEFAULT_TIME_SLEEP = 1

def fixar_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)

    # if torch.cuda.is_available():
    #     # torch.cuda.manual_seed(seed)
    #     # torch.cuda.manual_seed_all(seed)  # para múltiplas GPUs
    #     torch.backends.cudnn.deterministic = True
    #     torch.backends.cudnn.benchmark = False



LLMModelsType = Union[
    str,
    Tuple[str, Literal['ollama', 'openrouter']]
]

class LLMInference:
    def __init__(self, api_key: str, models: List[LLMModelsType] = [], models_params: List[Dict[str, str]] = None,
                 system_prompt: str = None,
                 structured_output_instructions: Dict[str, Any] = None,
                 use_cache: bool = True,
                 cache_interval:int = 10,
                 cache_path: str = CACHE_PATH,
                 default_model_type:Literal['ollama', 'openrouter'] = 'ollama',
                 fix_seed:int = 42,
                 host_ollama: str = HOST_OLLAMA,
                 default_time_sleep: int = DEFAULT_TIME_SLEEP):
        """
        Initialize the batch inference class using OpenAI client for OpenRouter.

        Args:
            api_key (str): Your OpenRouter API key.
            models (List[str]): List of model names to query.
            system_prompt (str): Common system prompt for all queries.
            structured_output_instructions (Dict[str, Any]): Instructions to guide the output structure.
        """

        self.models = models
        if models_params is None:
            self.models_params = [DEFAULT_MODEL_PARAMS]*len(models)
        else:
            self.models_params = models_params
        self.system_prompt = system_prompt
        self.structured_output_instructions = structured_output_instructions
        self.api_key = api_key

        if fix_seed is not None:
            fixar_seed( fix_seed )

        self.seed = fix_seed
        self.default_model_type = default_model_type
        self.use_cache = use_cache
        self.cache_interval = cache_interval
        self.cache_path = cache_path
        self.host_ollama = host_ollama
        self.cache_counter = 0
        self.default_time_sleep = default_time_sleep

        if os.path.exists(self.cache_path):
            with open(self.cache_path) as f:
                self.cache = json.load(f)
                print('cache lido', flush=True)
        else:
            self.cache = {}

        weakref.finalize(self, self._save_cache)


    def _create_messages(self, user_prompt: str) -> List[Dict[str, str]]:
        """
        Create the message payload.

        Args:
            user_prompt (str): The user-specific prompt.

        Returns:
            List[Dict[str, str]]: Messages to send to the model.
        """
        messages = []
        if self.system_prompt:
            messages.append( {"role": "system", "content": self.system_prompt} )
        messages.append({"role": "user", "content": user_prompt})
        if self.structured_output_instructions:
            messages.append( {"role": "user", "content": f"Format your output according to: {self.structured_output_instructions}"} )
        return messages

    def _query_model_raw_openrouter(self, model: str, model_params: Dict[str,Any], user_prompt: str, max_retries:int=5) -> str:
        """
        Query a single model using OpenAI client.

        Args:
            model (str): Model name.
            user_prompt (str): User-specific prompt.

        Returns:
            str: Model's output content.
        """
        # completion = self.client.chat.completions.create(
        #     model=model,
        #     messages=self._create_messages(user_prompt),
        #     top_k=1,
        #     temperature=0.1
        # )
        # return completion.choices[0].message.content

        url = "https://openrouter.ai/api/v1/chat/completions"
        headers = {
        "Authorization": f"Bearer {self.api_key}",
        "Content-Type": "application/json"
        }
        payload = {
        "model": model,
        "messages": self._create_messages(user_prompt)
        }
        payload.update(model_params)


        retry_count = 0

        while True:
            try:
                response = requests.post(url, headers=headers, json=payload, timeout = 300)
                response.raise_for_status()  # levanta erro se status >= 400
                if self.default_time_sleep > 0:
                    time.sleep(self.default_time_sleep)
                # print("Sucesso:", response.status_code)
                break  # sai do loop se a requisição for bem-sucedida

            except (ConnectionError, RemoteDisconnected) as e:
                retry_count += 1
                if retry_count > max_retries:
                    print("Max retries reached. Exiting...")
                    raise
                print("Erro de conexao:", e)
                print(f"Erro de conexão detectado ({retry_count}/{max_retries}): {e}")
                print("Aguardando 60 segundos antes de tentar novamente...", flush=True)
                time.sleep(60)

            except Exception as e:
                print("Erro inesperado:", e)
                raise
                # break  # interrompe se for outro erro que não de conexão


        # response = requests.post(url, headers=headers, json=payload, timeout = 300)
        # response.raise_for_status()  # dispara exceção se status >= 400

        return response.json()['choices'][0]['message']['content']

    def _query_ollama(self, model: str, model_params: Dict[str, Any], user_prompt: str) -> str:
        if self.host_ollama == 'local':
            client = ollama
        else:
            client = ollama.Client(host=self.host_ollama)


        if self.seed is not None and not 'seed' in model_params:
            model_params['seed'] = self.seed

        response = client.generate(
            model=model,
            prompt=user_prompt,
            stream=False,
            options=model_params,
        )

        return response['response']
    def _save_cache( self ):
        with open(self.cache_path, "w") as f:
            json.dump(self.cache, f, indent=2)

    def query_model(self, model: str, model_params: Dict[str, Any], user_prompt: str, model_type:Literal['ollama', 'openrouter'] = None, max_retries:int=5 ) -> str:
        key = hashlib.md5(user_prompt.encode()+model.encode()+str(model_params).encode()).hexdigest()
        if key in self.cache:
            print( 'USed from cache.', flush=True)
            return self.cache[key]
        try:
            # response_TXT = call_ollama(MODEL, prompt)


            if model_type is None:
                model_type = self.default_model_type
            if model_type == 'ollama':
                response_TXT =  self._query_ollama(model, model_params, user_prompt)
            elif model_type == 'openrouter':
                continua = True
                while continua:
                    try:
                        response_TXT  = self._query_model_raw_openrouter(model, model_params, user_prompt, max_retries=max_retries)
                        continua = False
                    except requests.exceptions.HTTPError as e:
                        if e.response is not None and e.response.status_code == 429:
                            self._save_cache()
                            print("Erro 429: too many requests / rate limit exceeded. Sleeping...")
                            time.sleep( 60 )
                        else:
                            raise
            else:
                raise ValueError(f"Unsupported model type: {model_type}. Use 'ollama' or 'openrouter'.")
        except Exception as e:
            self._save_cache()
            raise

        reply = response_TXT.strip()
        self.cache[key] = reply
        self.cache_counter+=1
        if self.cache_counter % SAVE_CACHE_INTERVAL == 0:
            self._save_cache()
            print('Cache saved to disk.', flush=True)
        # print(f"Cache hit: {self.cache_counter} - {key}", flush=True)
        return reply

    def generate_outputs(self, dataset: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
        """
        Generate outputs for each model given a labeled dataset.

        Args:
            dataset (List[Dict[str, Any]]): List of data points with 'text' and 'label'.

        Returns:
            Dict[str, List[Dict[str, Any]]]: Outputs organized by model name.
        """
        all_outputs = {model: [] for model in self.models}

        for data_point in tqdm(dataset):
            user_prompt = f"Text: {data_point['text']}\nLabel: {data_point['label']}"

            for model, model_params in zip(self.models, self.models_params):
                try:
                    print(f"Querying model {model}...")
                    output = self.query_model(model, model_params, user_prompt)
                    all_outputs[model].append({
                        "input": data_point,
                        "output": output
                    })
                    time.sleep(1)  # Sleep between requests to be polite
                except Exception as e:
                    print(f"Error querying model {model}: {e}")
                    all_outputs[model].append({
                        "input": data_point,
                        "output": {"error": str(e)}
                    })

        return all_outputs

In [ ]:
DEFAULT_MODEL_PARAMS = {'top_k':1,'temperature':0.1, 'top_p':0.0001}

CACHE_PATH = "cache_llm_respostas_ANALISES.json"
#CACHE_PATH = "/content/drive/MyDrive/codigo_python/cache_llm_respostas_ANALISES.json"
# HOST_OLLAMA = "10.10.10.106:11434"
MY_KEY = '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'

my_llm = LLMInference(
            api_key = MY_KEY,
            use_cache = True,
            cache_interval = 10,
            cache_path = CACHE_PATH,
            default_model_type = 'openrouter', # 'ollama' ou 'openrouter'
            fix_seed = 42,
            host_ollama = 'local',
            default_time_sleep = 1
        )

cache lido


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt')
df.head(50)

,Index,Question,Proposed Solution,Answer
0,0,A population of protozoa grows at a constant r...,"We have dP/dt = kP Therefore dP/dt = 0,7944P ...",2.350000e+02
1,1,A tank contains 1000 L of salt water with 15 k...,Let S(t) be the amount of salt (in kilograms...,1.230000e+01
2,2,A resistor R=10Ω and an inductor L=2H are conn...,"Using Kirchhoff's Voltage Law KVL, the sum of ...",0.000000e+00
3,3,A 5μF capacitor is charged through a 10kΩ resi...,"In an RC charging circuit, Kirchhoff's voltag...",5.490000e-02
4,4,An object is removed from an oven at 200°C and...,Let T(t) represent the temperature of the obje...,1.141900e+02
5,5,A certain village in Angola had a population ...,Let P(t) be the population at time t. The rate...,9.260000e+00
6,6,"Find the charge when t=0,2, in a series RC cir...","We have Rdq(t)/dt+1/C q=V Since R=10, C=4*10^(...",-5.290000e-02
7,7,A resistor R=12Ω and an inductor L=4H are conn...,"Using Kirchhoff's Voltage Law (KVL), the sum o...",2.660000e-01
8,8,A roast chicken is removed from the oven when ...,Let T(t) represent the temperature of the roas...,1.364000e+02
9,9,"Escherichia coli, a common bacteria found in t...","We have dP/dt = kP. Integrating both sides, we...",1.676300e+02


In [ ]:
def ODE_modelling(model, text):
    prompt = f"""[Instruction]
You are Dr. ODELA – an expert in Ordinary Differential Equations (ODEs), with vast experience in formulating ODE models from natural language descriptions and in outputting the ODE model as Python code using either 'solve_ivp' or 'odeint' from SciPy.

Task 01:
Model the following problem using an ordinary differential equation (ODE).
Instructions:
1. Present the reasoning in a clear, step-by-step textual format.
2. Structure your response in natural language, suitable for conversion into Python code.
3. Convert all LaTeX-style math expressions into valid, readable mathematical expressions in plain text.
4. Do not compute final numerical answers; focus only on formulation and symbolic steps.

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Task 02:
Convert the given ODE model (from Task 01) into a complete and executable Python code that returns the final numerical answer. Use the most appropriate solver – either solve_ivp or odeint from SciPy – based on the characteristics of the ODE problem.
The Python code must:
1. Output only valid Python code, without explanations or comments.
2. Use either solve_ivp or odeint appropriately.
3. Treat all dependent variable inputs as one-element arrays when using solve_ivp to avoid shape mismatch errors.
4. Return only the final computed value; do not include intermediate steps.
5. Round the final result to exactly 5 decimal places.
6. Ensure the code runs without requiring any modifications.


Here comes 3  examples:

=====================================================================================================================================================================================================================================================================================================================================================================
[Example 1. A population of protozoa grows at a constant relative rate of 0.44 per individual per day. On day zero, the population consists of five  protozoa.Calculate the population size after 8 days?]

Here is  the ODE model and Python code that solves the given problem that meets the specified requirements:

  **Solution 1: Model the problem using an ODE**:

Step 1: Understand the nature of the Problem

We are told that the population of protozoa grows at a constant relative rate of 0.44 per individual per day. This indicates exponential growth, where the rate of change of the population is proportional to the current population size.

Let:

P(t) be the population size at time t (in days),

r = 0.44 be the relative growth rate per day.

Step 2.The differential equation that models the population growth is:

dP/dt = r * P

Substituting r = 0.44, we get:

dP/dt = 0.44 * P

Step 3.Initial condition:

At t = 0, the population is P(0) = 5.

Step 4.We are asked to find the population after 8 days, i.e., compute P(8)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
**Solution 02: Convert the ODE model into Python code**

```python
from scipy.integrate import solve_ivp
import numpy as np

def population_growth(t, y):
    return [0.44 * y[0]]

sol = solve_ivp(population_growth, [0, 8], [5], t_eval=[8])
result = round(sol.y[0][0], 5)
print(result)```

=================================================================================================================================================================================================================================================================================
[Example 2. A body whose temperature is 20° is placed in a medium which is kept at a constant temperature of 60°. In 5 minute, the body's temperature has risen to 30°. How long will it take for the body's temperature to increase to 40°?]

Here is  the ODE model and Python code that solves the given problem that meets the specified requirements:

**Solution 1: Model the problem using an ODE**:

1.Understand the problem:
A body with an initial temperature of 20°C is placed in a medium at a constant temperature of 60°C. After 5 minutes, the body's temperature rises to 30°C. We need to model how the body's temperature changes over time and determine how long it takes to reach 40°C.

2. Newton's Law of Cooling/Warming:
The rate of temperature change of the body is proportional to the difference between the medium's temperature (T_medium=60°C) and the body's temperature (T(t)). The ODE is: dT/dt =k(60−T).
In plain text: dT/dt = k * (60 - T)
3. Initial condition:
At t=0, T(0)=20
4. Additional condition for solving k:
At t=5, T(5)=30
5. Solution approach:
The ODE is separable and has an analytical solution, but we will solve it numerically to find the time when T=40°C.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
**Solution 02: Convert the ODE model into Python code**

```python
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve

def model(t, T, k):
    return k * (60 - T)

def find_k(k_guess):
    sol = solve_ivp(model, (0, 5), [20], args=(k_guess,), t_eval=[5])
    return sol.y[0][-1] - 30

k = fsolve(find_k, 0.1)[0]
sol = solve_ivp(model, (0, 100), [20], args=(k,), t_eval=np.linspace(0, 100, 1000))
target_temp = 40
time_idx = np.argmin(np.abs(sol.y[0] - target_temp))
result = round(sol.t[time_idx], 5)
print(result)```

=======================================================================================================================================================================================
[Example 3. A 6μF capacitor is charged through a 30kΩ resistor from a 48V battery. Calculate how long will
it take for the voltage across the capacitor to reach 12V ?]

Here is  the ODE model and Python code that solves the given problem that meets the specified requirements:

**Solution 1: Model the problem using an ODE**:

1.Understand the problem:
We have an RC circuit with:

Capacitance C = 6μF (6×10⁻⁶ F)

Resistance R = 30kΩ (30×10³ Ω)

Battery voltage V₀ = 48V
We need to find the time it takes for the capacitor voltage to reach 12V.

2.RC circuit behavior:
The voltage V across a charging capacitor follows:
dV/dt = (V₀ - V)/(R×C)
In plain text: dV/dt = (V0 - V)/(R*C)

3.Initial condition:
At t = 0, V(0) = 0 (assuming capacitor starts uncharged)

4.Solution approach:
While this has an analytical solution (V(t) = V₀(1 - e^(-t/RC))),
we'll solve numerically to demonstrate the ODE approach.
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
**Solution 02: Convert the ODE model into Python code**

```python
import numpy as np
from scipy.integrate import solve_ivp

def model(t, V, R, C, V0):
    return (V0 - V)/(R*C)

R = 30e3
C = 6e-6
V0 = 48
target_V = 12
sol = solve_ivp(model, [0, 10], [0], args=(R, C, V0), dense_output=True)
t = np.linspace(0, 10, 1000)
V = sol.sol(t)[0]
time = t[np.argmin(np.abs(V - target_V))]
print(round(float(time), 5))```

======================================================================================================================================================================================================================================
The following sentences  are forbidden:
”””
This code solves the  problem for the given differential equation using 'solve_ivp' from SciPy;
This code solves the  problem for the given differential equation using ‘odeint‘ from SciPy.”””

Please avoid the above sentences.
======================================================================================================================================================================================================================================
Take it easy and have a deep breath before answering the question. This won't be a challenge for you atall.


Here comes the question:
{text}

Your response:
"""
    response = my_llm.query_model( model, DEFAULT_MODEL_PARAMS, prompt)
    print("**************", model)
    print(response)
    print("************")
    return response

In [ ]:
text = df.iloc[0]['Question']
model = 'deepseek/deepseek-r1-distill-llama-70b'
ODE_modelling(model, text)

USed from cache.
************** deepseek/deepseek-r1-distill-llama-70b
**Solution 1: Model the problem using an ODE**

Step 1: Understand the nature of the Problem

We are dealing with a population of protozoa that grows at a constant relative rate of 0.7944 per individual per day. This indicates exponential growth, where the rate of change of the population is proportional to the current population size.

Let:

P(t) be the population size at time t (in days),

r = 0.7944 be the relative growth rate per day.

Step 2. The differential equation that models the population growth is:

dP/dt = r * P

Substituting r = 0.7944, we get:

dP/dt = 0.7944 * P

Step 3. Initial condition:

At t = 0, the population is P(0) = 2.

Step 4. We are asked to find the population after 6 days, i.e., compute P(6)

```python
from scipy.integrate import solve_ivp
import numpy as np

def population_growth(t, y):
    return [0.7944 * y[0]]

sol = solve_ivp(population_growth, [0, 6], [2], t_eval=[6])
result = roun

'**Solution 1: Model the problem using an ODE**\n\nStep 1: Understand the nature of the Problem\n\nWe are dealing with a population of protozoa that grows at a constant relative rate of 0.7944 per individual per day. This indicates exponential growth, where the rate of change of the population is proportional to the current population size.\n\nLet:\n\nP(t) be the population size at time t (in days),\n\nr = 0.7944 be the relative growth rate per day.\n\nStep 2. The differential equation that models the population growth is:\n\ndP/dt = r * P\n\nSubstituting r = 0.7944, we get:\n\ndP/dt = 0.7944 * P\n\nStep 3. Initial condition:\n\nAt t = 0, the population is P(0) = 2.\n\nStep 4. We are asked to find the population after 6 days, i.e., compute P(6)\n\n```python\nfrom scipy.integrate import solve_ivp\nimport numpy as np\n\ndef population_growth(t, y):\n    return [0.7944 * y[0]]\n\nsol = solve_ivp(population_growth, [0, 6], [2], t_eval=[6])\nresult = round(sol.y[0][0], 5)\nprint(result)\n``

In [ ]:
import pandas as pd
import os
import hashlib
import json
from pathlib import Path

# Configuração do cache
# CACHE_DIR = "cache"
# CACHE_FILE = Path(CACHE_DIR) / "ode_cache.json"

# Garante que o diretório de cache existe
# os.makedirs(CACHE_DIR, exist_ok=True)

CACHE_FILE = Path('/content/drive/MyDrive/math_ODEsdataset_csv.txt_ler')

# Tenta carregar o cache existente
cache = {}
if CACHE_FILE.exists():
    print( 'Cache lido dos codigos!!!' )
    try:
        with open(CACHE_FILE, 'r') as f:
            cache = json.load(f)
        print(f"Cache carregado com {len(cache)} entradas")
    except:
        print("Erro ao carregar cache, iniciando novo")

def get_cache_key(model, question):
    """Gera chave única para cache"""
    return hashlib.md5(f"{model}_{question}".encode()).hexdigest()

# Carrega o dataset
df = pd.read_csv('/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt')

# Processa cada modelo
for model in [

           'deepseek/deepseek-r1-distill-llama-70b'





]:
    responses = []
    print(f"\n{'='*50}")
    print(f"Processando modelo: {model}")
    print(f"{'='*50}")

    # Verifica se já temos respostas para este modelo
    model_col = model + '_class'
    if model_col in df.columns:
        existing_responses = df[model_col].tolist()
    else:
        existing_responses = [None] * len(df)

    for index, row in df.iterrows():
        question = str(row['Question'])
        cache_key = get_cache_key(model, question)

        # Usa resposta em cache se disponível
        if cache_key in cache:
            responses.append(cache[cache_key])
            print(f"✓ Usando cache para índice {index}")
            continue

        # Usa resposta existente se válida
        if existing_responses[index] not in [None, 'error']:
            responses.append(existing_responses[index])
            cache[cache_key] = existing_responses[index]  # Adiciona ao cache
            print(f"↻ Reutilizando resposta existente para índice {index}")
            continue

        # Processa nova resposta
        max_tries = 3
        tries = 0
        result = 'error'

        while tries < max_tries:
            try:
                rsp = ODE_modelling(model, question)
                if rsp != 'error':
                    result = rsp
                    break
            except Exception as e:
                print(f"Erro inesperado: {e}")

            tries += 1
            print(f"↻ Tentativa {tries}/{max_tries} falhou no índice {index}")

        responses.append(result)
        cache[cache_key] = result  # Atualiza cache

        # Salva cache periodicamente
        if index % 10 == 0:
            with open(CACHE_FILE, 'w') as f:
                json.dump(cache, f)
            print(f"💾 Cache salvo após índice {index}")

    df[model_col] = responses

    # Salva cache após cada modelo
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f)
    print(f"💾 Cache salvo após processar modelo {model}")

# Garante que a pasta 'results' existe
os.makedirs('results', exist_ok=True)

# Salva o CSV final
df.to_csv('/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models', index=False)
print("✅ Processamento completo! Resultados salvos.")

Cache lido dos codigos!!!
Cache carregado com 1274 entradas

Processando modelo: deepseek/deepseek-r1-distill-llama-70b
✓ Usando cache para índice 0
✓ Usando cache para índice 1
✓ Usando cache para índice 2
✓ Usando cache para índice 3
✓ Usando cache para índice 4
✓ Usando cache para índice 5
✓ Usando cache para índice 6
✓ Usando cache para índice 7
✓ Usando cache para índice 8
✓ Usando cache para índice 9
✓ Usando cache para índice 10
✓ Usando cache para índice 11
✓ Usando cache para índice 12
✓ Usando cache para índice 13
✓ Usando cache para índice 14
✓ Usando cache para índice 15
✓ Usando cache para índice 16
✓ Usando cache para índice 17
✓ Usando cache para índice 18
✓ Usando cache para índice 19
✓ Usando cache para índice 20
✓ Usando cache para índice 21
✓ Usando cache para índice 22
✓ Usando cache para índice 23
✓ Usando cache para índice 24
✓ Usando cache para índice 25
✓ Usando cache para índice 26
✓ Usando cache para índice 27
✓ Usando cache para índice 28
✓ Usando cache para 

In [ ]:
model_csv=pd.read_csv('/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models')
model_csv.head(50)

,Index,Question,Proposed Solution,Answer,deepseek/deepseek-r1-distill-llama-70b_class
0,0,A population of protozoa grows at a constant r...,"We have dP/dt = kP Therefore dP/dt = 0,7944P ...",2.350000e+02,**Solution 1: Model the problem using an ODE**...
1,1,A tank contains 1000 L of salt water with 15 k...,Let S(t) be the amount of salt (in kilograms...,1.230000e+01,To determine the amount of salt remaining in t...
2,2,A resistor R=10Ω and an inductor L=2H are conn...,"Using Kirchhoff's Voltage Law KVL, the sum of ...",0.000000e+00,**Solution 1: Model the problem using an ODE**...
3,3,A 5μF capacitor is charged through a 10kΩ resi...,"In an RC charging circuit, Kirchhoff's voltag...",5.490000e-02,**Solution 1: Model the problem using an ODE**...
4,4,An object is removed from an oven at 200°C and...,Let T(t) represent the temperature of the obje...,1.141900e+02,**Solution 1: Model the problem using an ODE**...
5,5,A certain village in Angola had a population ...,Let P(t) be the population at time t. The rate...,9.260000e+00,**Solution 1: Model the problem using an ODE**...
6,6,"Find the charge when t=0,2, in a series RC cir...","We have Rdq(t)/dt+1/C q=V Since R=10, C=4*10^(...",-5.290000e-02,**Solution 1: Model the problem using an ODE**...
7,7,A resistor R=12Ω and an inductor L=4H are conn...,"Using Kirchhoff's Voltage Law (KVL), the sum o...",2.660000e-01,**Solution 1: Model the problem using an ODE**...
8,8,A roast chicken is removed from the oven when ...,Let T(t) represent the temperature of the roas...,1.364000e+02,**Solution 1: Model the problem using an ODE**...
9,9,"Escherichia coli, a common bacteria found in t...","We have dP/dt = kP. Integrating both sides, we...",1.676300e+02,**Solution 1: Model the problem using an ODE**...


In [ ]:
model_csv.iloc[37]['deepseek/deepseek-r1-distill-llama-70b_class']

"**Solution 1: Model the problem using an ODE**\n\nStep 1: Understand the problem\n\nA man jumps from an airplane and opens his parachute after 10 seconds. We need to find his velocity at 15 seconds. The forces acting on him are gravity and air resistance, which is proportional to his velocity. The resistance is different when the parachute is closed (first 10 seconds) and open (after 10 seconds).\n\nStep 2: Define variables and constants\n\nLet:\n- \\( v(t) \\) be the velocity at time \\( t \\) in seconds.\n- \\( m = 5 \\) slugs (mass, calculated from weight \\( W = 160 \\) lbf and \\( g = 32 \\) ft/s²).\n- \\( g = 32 \\) ft/s² (acceleration due to gravity).\n- Air resistance coefficients: \\( k_1 = 0.5 \\) lbf-s/ft when parachute is closed, \\( k_2 = 10 \\) lbf-s/ft when open.\n\nStep 3: Formulate the ODE\n\nThe net force on the man is given by Newton's second law:\n\\[ m \\frac{dv}{dt} = mg - k v \\]\n\nThis leads to two ODEs depending on the time interval:\n- For \\( t < 10 \\): \\

In [ ]:
model_csv.iloc[5]['Proposed Solution']

'Let P(t) be the population at time t. The rate of population decrease is proportional to the population: dP/dt=-kP where k is the decay constant. Solving this ODE,  we have P(t)=Ae^(-kt). Applying initial condition, P(0)=10,000, we have A=10,000 → P(t)=10,000e^(-kt). After 3 years, the population is 8,000. Substituting  P(3)=8,000 into the equation and calculating the value of k, we have 8,000/10,000=e^(-3k)→ ln(0.8)=-3k  → k=−ln(0.8)/3 ≈ 0.0749 . Hence P(t)=10,000e^(-0,0749t) .Now setting  P(t)=5,000 and solving for t, we have  5,000 =10,000e^(-0,0749t)→ 5,000/10,000=e^(-0,0749t)   → t=−ln(0.5)/0,0749≈ 9.26  years. Therefore it will take approximately 9.26 years for the population to reduce to 5,000 people'

In [ ]:
model_csv.iloc[5]['Question']

'A certain village in Angola  had a population of 10,000 people. Due to unfavorable living conditions, the population decreases at a rate proportional to the current population. After 3 years, the population  decreased to 8,000. Determine the time it will take for the population to reduce to 5,000?'

In [ ]:
import pandas as pd
from tqdm import tqdm
import re
import io
import sys

# Função para extrair blocos de código Python
def extract_python_code(text):
    """
    Extrai blocos de código Python delimitados por ```Python ou ```python.
    """
    pattern = r'```(?:Python|python)(.*?)```'
    code_blocks = re.findall(pattern, text, re.DOTALL)
    return [block.strip() for block in code_blocks]

# Função para executar código Python e capturar o último valor numérico
def execute_python_code(code):
    local_vars = {}
    try:
        exec(code, {}, local_vars)  # executa em ambiente isolado
        numeric_values = [v for v in local_vars.values() if isinstance(v, (int, float))]
        return numeric_values[-1] if numeric_values else "erro (nenhum valor numérico encontrado)"
    except Exception:
        return "erro"

# Carrega o DataFrame
model_results = pd.read_csv('/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models')
# Lista de colunas de modelos
model_columns = ['openai/gpt-5-chat_class','meta-llama/llama-3.1-70b-instruct_class','deepseek/deepseek-r1-distill-llama-70b_class','qwen/qwen-max_class','mistralai/mixtral-8x22b-instruct_class' ]

# Processa cada coluna se ela existir no CSV
for model_col in tqdm(model_columns, desc="Processando modelos"):
    if model_col not in model_results.columns:
        print(f"Aviso: coluna '{model_col}' não encontrada no CSV. Pulando.")
        continue

    results = []
    for text in tqdm(df[model_col], desc=f"Processando {model_col}", leave=False):
        code_blocks = extract_python_code(str(text))
        if not code_blocks:
            results.append("erro (sem código)")
            continue

        final_result = "erro"
        for code in code_blocks:
            result = execute_python_code(code)
            if isinstance(result, (int, float)):  # achou número
                final_result = result
                break

        results.append(final_result)

    # Adiciona coluna de resultados
    model_results[f"{model_col}_result"] = results

# Salva o DataFrame com resultados
output_path = '/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models_final_results.csv'
model_results.to_csv(output_path, index=False)
print(f"Processamento concluído. Resultados salvos em: {output_path}")


Processando modelos:   0%|          | 0/5 [00:00<?, ?it/s]

Aviso: coluna 'openai/gpt-5-chat_class' não encontrada no CSV. Pulando.
Aviso: coluna 'meta-llama/llama-3.1-70b-instruct_class' não encontrada no CSV. Pulando.



Processando deepseek/deepseek-r1-distill-llama-70b_class:   0%|          | 0/50 [00:00<?, ?it/s]
Processando deepseek/deepseek-r1-distill-llama-70b_class:  92%|█████████▏| 46/50 [00:00<00:00, 396.49it/s]
Processando modelos: 100%|██████████| 5/5 [00:00<00:00, 35.33it/s]

234.9685
12.28096
0.0
0.06006
136.92926
17.31732
81.11965
18.52046
21.98198
15.53554
97.06962
15.53554
2e-05
55.71343
4.99809
76.03554
73598.56979
39.87399
13.51351
17.33734
1.998
12.51454
40833.33598
27.74775
16.0095
1.19957
7303.28128
0.07232
3.99925
130.0
0.01001
11.62841
4.96571
Aviso: coluna 'qwen/qwen-max_class' não encontrada no CSV. Pulando.
Aviso: coluna 'mistralai/mixtral-8x22b-instruct_class' não encontrada no CSV. Pulando.
Processamento concluído. Resultados salvos em: /content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models_final_results.csv


In [ ]:
results_final=pd.read_csv('/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models_final_results.csv')
results_final.head(50)

,Index,Question,Proposed Solution,Answer,deepseek/deepseek-r1-distill-llama-70b_class,deepseek/deepseek-r1-distill-llama-70b_class_result
0,0,A population of protozoa grows at a constant r...,"We have dP/dt = kP Therefore dP/dt = 0,7944P ...",2.350000e+02,**Solution 1: Model the problem using an ODE**...,234.9685
1,1,A tank contains 1000 L of salt water with 15 k...,Let S(t) be the amount of salt (in kilograms...,1.230000e+01,To determine the amount of salt remaining in t...,12.28096
2,2,A resistor R=10Ω and an inductor L=2H are conn...,"Using Kirchhoff's Voltage Law KVL, the sum of ...",0.000000e+00,**Solution 1: Model the problem using an ODE**...,0.0
3,3,A 5μF capacitor is charged through a 10kΩ resi...,"In an RC charging circuit, Kirchhoff's voltag...",5.490000e-02,**Solution 1: Model the problem using an ODE**...,0.06006006006006006
4,4,An object is removed from an oven at 200°C and...,Let T(t) represent the temperature of the obje...,1.141900e+02,**Solution 1: Model the problem using an ODE**...,erro
5,5,A certain village in Angola had a population ...,Let P(t) be the population at time t. The rate...,9.260000e+00,**Solution 1: Model the problem using an ODE**...,erro
6,6,"Find the charge when t=0,2, in a series RC cir...","We have Rdq(t)/dt+1/C q=V Since R=10, C=4*10^(...",-5.290000e-02,**Solution 1: Model the problem using an ODE**...,erro
7,7,A resistor R=12Ω and an inductor L=4H are conn...,"Using Kirchhoff's Voltage Law (KVL), the sum o...",2.660000e-01,**Solution 1: Model the problem using an ODE**...,erro
8,8,A roast chicken is removed from the oven when ...,Let T(t) represent the temperature of the roas...,1.364000e+02,**Solution 1: Model the problem using an ODE**...,136.92926
9,9,"Escherichia coli, a common bacteria found in t...","We have dP/dt = kP. Integrating both sides, we...",1.676300e+02,**Solution 1: Model the problem using an ODE**...,erro


In [ ]:
results_final.iloc[1][['Proposed Solution','deepseek/deepseek-r1-distill-llama-70b_class','Answer', 'deepseek/deepseek-r1-distill-llama-70b_class_result']]


,1
Proposed Solution,Let S(t) be the amount of salt (in kilograms...
deepseek/deepseek-r1-distill-llama-70b_class,To determine the amount of salt remaining in t...
Answer,12.3
deepseek/deepseek-r1-distill-llama-70b_class_result,12.28096


In [ ]:
import pandas as pd
import numpy as np

# Configurações de tolerância
ABS_TOL = 0.1     # Tolerância absoluta
REL_TOL = 0.05    # Tolerância relativa

# Caminho do arquivo
csv_path = '/content/drive/MyDrive/math_ODEsdataset_csv_can llm solve ode.txt_models_final_results.csv'

# Carrega o DataFrame
df = pd.read_csv(csv_path)

# Verifica se coluna 'Answer' existe
if "Answer" not in df.columns:
    raise ValueError(f"O arquivo {csv_path} não contém a coluna 'Answer'. Colunas encontradas: {df.columns.tolist()}")

# Identifica colunas de resultado dos modelos
model_cols = [col for col in df.columns if col.endswith('_exec_result') or col.endswith('_result')]

results = []
for model in model_cols:
    correct = 0
    total = len(df)
    valid_samples = 0

    for _, row in df.iterrows():
        truth = row["Answer"]
        pred = row[model]

        # Ignora previsões com erro
        if isinstance(pred, str) and ("erro" in pred.lower() or "error" in pred.lower()):
            continue

        try:
            # Converte para float
            truth_val = float(str(truth).replace(',', '.'))
            pred_val = float(str(pred).replace(',', '.'))
            valid_samples += 1

            diff = abs(pred_val - truth_val)
            rel_diff = diff / max(abs(truth_val), 1e-8)

            if diff <= ABS_TOL or rel_diff <= REL_TOL:
                correct += 1

        except (ValueError, TypeError):
            continue

    accuracy = round((correct / total) * 100, 5) if total > 0 else 0.0

    results.append({
        'Modelo': model.replace('_exec_result', '').replace('_result', ''),
        'Amostras válidas': valid_samples,
        'Total de amostras': total,
        'Acurácia (%)': accuracy
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Acurácia (%)", ascending=False)

print("\n📊 Acurácia dos modelos:")
print(results_df.to_string(index=False))



📊 Acurácia dos modelos:
                                      Modelo  Amostras válidas  Total de amostras  Acurácia (%)
deepseek/deepseek-r1-distill-llama-70b_class                33                 50          60.0
